In [ ]:
import pickle
import random

import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import scipy as sc
from scipy.spatial import Delaunay

from simplicial_kuramoto import SimplicialComplex, plotting
from simplicial_kuramoto.graph_generator import modular_graph
from simplicial_kuramoto.integrators import *

In [ ]:
def get_grid(n_node):

    x = np.linspace(0, 1, n_node)

    points = []
    for i in range(n_node):
        for j in range(n_node):
            points.append([x[j], x[i]])

    tri = Delaunay(points)

    edge_list = []
    for t in tri.simplices:
        edge_list.append([t[0], t[1]])
        edge_list.append([t[0], t[2]])
        edge_list.append([t[1], t[2]])

    graph = nx.Graph()
    graph.add_nodes_from(np.arange(len(points)))
    graph.add_edges_from(edge_list)
    return graph, points

In [ ]:
graph, points = get_grid(5)

plt.figure(figsize=(10, 10))
nx.draw_networkx_nodes(graph, pos=points, node_size=20)
nx.draw_networkx_edges(graph, pos=points)

labels = dict(((u, v), d) for d, (u, v) in enumerate(graph.edges))
nx.draw_networkx_edge_labels(graph, pos=points, edge_labels=labels)

# Fixed number of faces, changing initial conditions

In [ ]:
def complex_info(complex_delaunay, edge_result):
    B0 = complex_delaunay.node_incidence_matrix
    W0 = complex_delaunay.node_weights_matrix
    B1 = complex_delaunay.edge_incidence_matrix
    W1 = complex_delaunay.edge_weights_matrix
    W2 = complex_delaunay.face_weights_matrix
    omega_0 = np.zeros(complex_delaunay.n_edges)

    Ld = W1.dot(B0.dot(W0.dot(B0.T)))
    Lu = W1.dot(B1.T.dot(W2.dot(B1)))

    L1 = Ld + Lu

    mod_sol = np.mod(
        np.around(L1.dot(edge_result.y[:, -1]), 10), np.around(2 * np.pi, 10)
    )
    print(mod_sol)
    print(
        np.mod(
            np.around(L1.dot(edge_result.y[:, -1]).sum(), 10), np.around(2 * np.pi, 10)
        )
    )

    print(sc.linalg.null_space(L1.todense()).shape[1])

In [ ]:
mu = np.pi
sigma = np.linspace(0, 0.01, 10)

t_max = 100
n_t = 10

all_cliques = nx.enumerate_all_cliques(graph)
faces = [clique for clique in all_cliques if len(clique) == 3]

n_repeats = 5
N_faces = [0, 11, 22, 32]
N_flip = 0

order = np.zeros([len(sigma), len(N_faces), n_repeats])

for si, s in enumerate(sigma):
    print(s)
    for nfi, nf in enumerate(N_faces):
        print(nf)
        for k in range(0, n_repeats):
            initial_phase = np.random.normal(mu, s, len(graph.edges))
            #             randomly flip 10 edges
            for j in range(N_flip):
                print(j)
                initial_phase[np.random.randint(1, len(graph.edges))] = np.pi

            rand_faces = random.sample(faces, int(nf))
            complex_delaunay = SimplicialComplex(
                graph=graph, no_faces=False, faces=rand_faces
            )

            edge_result = integrate_edge_kuramoto(
                complex_delaunay, initial_phase, t_max, n_t
            )

            #             complex_info(complex_delaunay,edge_result)

            op = plotting.plot_order_parameter(
                edge_result.y, return_op=True, plot=False
            )
            order[si, nfi, k] = op[-1]
#             order_repeat.append(op[-1])

#         order_sigma.append(order_repeat)
#     order.append(order_sigma)

In [ ]:
plt.plot(sigma, np.squeeze(order[:, 0, :]), "b*-")

In [ ]:
# mean_order = np.array([np.mean(l) for l in order])
# std_order = np.array([np.std(l) for l in order])
for nfi, nf in enumerate(N_faces):
    mean_order = np.mean(np.squeeze(order[:, nfi, :]), 1)
    std_order = np.std(np.squeeze(order[:, nfi, :]), 1)

    fig, ax = plt.subplots(1)
    ax.plot(sigma, mean_order, lw=2, label="Mean Order", color="blue")
    ax.fill_between(
        sigma,
        mean_order + std_order,
        mean_order - std_order,
        facecolor="blue",
        alpha=0.5,
    )
    ax.set_title(
        "Increasing sigma for N_faces "
        + str(nf)
        + " and "
        + str(N_flip)
        + ' "flipped" edges'
    )
    ax.legend(loc="upper right")
    ax.set_xlabel("Sigma")
    ax.set_ylabel("Order")
    ax.grid()
    plt.savefig(str(nf) + ".png")